In [4]:
# Importing necessary libraries
from matplotlib import legend
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import nn
import os
import numpy as np
from icecream import ic
from tqdm import tqdm
from itertools import chain
import logging
import os
import cProfile
import pstats
#from dataloader import *
from get_data import *
from torch.utils.data import Dataset, DataLoader

In [ ]:
visualise(get_data(path = "save_data_test3.csv", timesteps_from_data=0, skip_steps_start = 0, skip_steps_end = 0, 
                   drop_half_timesteps = False, normalise_s_w=False, rescale_p=False, num_inits=100), num_inits=10, set_ylim=True)

In [3]:
ws=50
n = 5


data = get_data(path = "save_data_test3.csv", timesteps_from_data=0, skip_steps_start = 0, skip_steps_end = 400, drop_half_timesteps = True, normalise_s_w=False, rescale_p=False, num_inits=n)
data_set = CustomDataset(data, window_size=ws, future=3)
dataloader = DataLoader(data_set, batch_size=1)

time = np.linspace(0,data.size(dim=1)*n,data.size(dim=1)*n)

for i, (inp, label) in enumerate(dataloader):
    if i%2==0:
     plt.plot(time[i:i+ws], inp[0,:,0].numpy(), linewidth=5, alpha=0.2, color="red")
     #plt.scatter(time[i:i+ws], label[0,:,0].numpy(), linewidth=5, alpha=0.3, color="red")
     
    else:
     plt.plot(time[i:i+ws], inp[0,:,0].numpy(), linewidth=5, alpha=0.2, color="blue")

plt.grid()
plt.show()



NameError: name 'get_data' is not defined

In [5]:
class CustomDataset(Dataset):
    def __init__(self, data, window_size, future=1):

        self.data = data
        self.ws = window_size
        self.future = future

    def __len__(self):
        return self.data.size(0)*self.data.size(1) - (self.ws + 1) - self.future

    def __getitem__(self, idx):

        j = int(idx/self.data.size(1))  

        k = int((idx +  self.ws) / self.data.size(1))

        m = (idx + self.ws) - k * self.data.size(1)

        index = idx % self.data.size(1)

        if j < k :

            print("padded label")
            inp=torch.cat((self.data[j, index : self.data.size(1) , :],
                          self.data[j, self.data.size(1) - 1, :].repeat(m, 1)))
            
            if self.future>1:
                label = self.data[j, (k * self.data.size(1))%self.data.size(1) - 1 : (k * self.data.size(1))%self.data.size(1) - 1 + self.future, :]         
            else:
                print("dont go here")
                label = self.data[j,(k * self.data.size(1))%self.data.size(1) - 1 , :]
            
        else:

            inp = self.data[j, index : index + self.ws, :]

            if self.future>1:
                print("regular label")
                label = self.data[j, index + self.ws : index + self.ws + self.future  , :]
            else:
                print("dont go here")
                label = self.data[j, index + self.ws, :]

        return inp, label

    def get_all_data(self):
        return self.data

In [6]:
ws= 3
n = 2

data = get_data(path = "save_data_test3.csv", timesteps_from_data=0, skip_steps_start = 0, skip_steps_end = 1000, drop_half_timesteps = True, normalise_s_w=False, rescale_p=False, num_inits=n)
data_set = CustomDataset(data, window_size=ws, future=1)
dataloader = DataLoader(data_set, batch_size=2)

time = np.linspace(0,data.size(dim=1)*n,data.size(dim=1)*n)
print(data.size())
for i, (inp, label) in enumerate(dataloader):
    #print("input" , inp)
    #print("label",label)
    print(i, label.size())

torch.Size([2, 50, 3])
dont go here
dont go here
0 torch.Size([2, 3])
dont go here
dont go here
1 torch.Size([2, 3])
dont go here
dont go here
2 torch.Size([2, 3])
dont go here
dont go here
3 torch.Size([2, 3])
dont go here
dont go here
4 torch.Size([2, 3])
dont go here
dont go here
5 torch.Size([2, 3])
dont go here
dont go here
6 torch.Size([2, 3])
dont go here
dont go here
7 torch.Size([2, 3])
dont go here
dont go here
8 torch.Size([2, 3])
dont go here
dont go here
9 torch.Size([2, 3])
dont go here
dont go here
10 torch.Size([2, 3])
dont go here
dont go here
11 torch.Size([2, 3])
dont go here
dont go here
12 torch.Size([2, 3])
dont go here
dont go here
13 torch.Size([2, 3])
dont go here
dont go here
14 torch.Size([2, 3])
dont go here
dont go here
15 torch.Size([2, 3])
dont go here
dont go here
16 torch.Size([2, 3])
dont go here
dont go here
17 torch.Size([2, 3])
dont go here
dont go here
18 torch.Size([2, 3])
dont go here
dont go here
19 torch.Size([2, 3])
dont go here
dont go here
2

In [13]:
a,b = next(iter(dataloader))
c, d = next(iter(dataloader))

print(a,b)
print(c,d)

dont go here
dont go here
dont go here
dont go here
tensor([[[2.2950e+05, 2.6985e-04, 0.0000e+00],
         [2.3104e+05, 2.6993e-04, 7.9591e-03],
         [2.3258e+05, 2.7016e-04, 1.4714e-02]],

        [[2.3104e+05, 2.6993e-04, 7.9591e-03],
         [2.3258e+05, 2.7016e-04, 1.4714e-02],
         [2.3412e+05, 2.7052e-04, 2.2013e-02]]], dtype=torch.float64) tensor([[2.3412e+05, 2.7052e-04, 2.2013e-02],
        [2.3566e+05, 2.7104e-04, 3.0136e-02]], dtype=torch.float64)
tensor([[[2.2950e+05, 2.6985e-04, 0.0000e+00],
         [2.3104e+05, 2.6993e-04, 7.9591e-03],
         [2.3258e+05, 2.7016e-04, 1.4714e-02]],

        [[2.3104e+05, 2.6993e-04, 7.9591e-03],
         [2.3258e+05, 2.7016e-04, 1.4714e-02],
         [2.3412e+05, 2.7052e-04, 2.2013e-02]]], dtype=torch.float64) tensor([[2.3412e+05, 2.7052e-04, 2.2013e-02],
        [2.3566e+05, 2.7104e-04, 3.0136e-02]], dtype=torch.float64)


In [ ]:


import matplotlib.pyplot as plt
import numpy as np

# Generate some sample data
x = np.arange(10)  # Extend the range of x-values
y1 = np.array([1, 2, 3, 4])
y2 = np.array([2, 3, 4, 5])
y3 = np.array([3, 4, 5, 6])

# Plot each array with offset
plt.plot(x[:4], y1, label='Array 1')
plt.plot(x[1:5], y2, label='Array 2')
plt.plot(x[2:6], y3, label='Array 3')
plt.xlim(0,10)
# Add labels and legend
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.title('Arrays Offset Plot')
plt.legend()

# Show plot
plt.grid(True)
plt.show()